In [ ]:
from models import MNISTConvNet, MNISTConvNetTrainer, get_mnist_dataloader
import matplotlib.pyplot as plt

train, val, test = get_mnist_dataloader(128)

# print some sample images from the training set with matplotlib
fig, axs = plt.subplots(2, 5, figsize=(12, 3))
for i, ax in enumerate(axs.flat):
    ax.imshow(train.dataset[i][0].squeeze(), cmap='gray')
    ax.set_title(f'Label: {train.dataset[i][1]}')

model = MNISTConvNet()
trainer = MNISTConvNetTrainer(model, train, val, device='mps')

trainer.train(num_epochs=10)

In [ ]:
# save the model
